Este notebook está vinculado al caso de estudio "Yapo" (ver el [documento](https://docs.google.com/document/d/1SwhwMOyG7-cmzasDeisQZ_NzX5v2VoZ7--lI4UhJz_U/edit#heading=h.5qaqs0frqkx4) de presentación). Se busca construir una base de datos de anuncios en Los Rios recopilando algunos anuncios de Yapo.cl.


# 1. Construcción de la base de datos

In [ ]:
import mysql.connector 

db_connection = mysql.connector.connect(user="root",host="localhost",password="root")
cursor = db_connection.cursor()

In [ ]:
cursor.execute("DROP DATABASE Yapo2;")

In [ ]:
cursor.execute("CREATE DATABASE Yapo2;")
cursor.execute("USE Yapo2")

#tabla vendedor
cursor.execute("CREATE TABLE vendedor ("+
               "id_vendedor VARCHAR(130) PRIMARY KEY, nombre VARCHAR(100), ciudad VARCHAR(30), codigo_region VARCHAR(7), "+
               "nombre_region VARCHAR(30), "+
               "fecha_inscripcion DATE)")

#tabla anuncio
cursor.execute("CREATE TABLE anuncio (url VARCHAR(300) PRIMARY KEY, "+
               "titulo VARCHAR(200), descripcion MEDIUMTEXT, precio INT, moneda VARCHAR(2), fecha_publicacion DATE, categoria VARCHAR(50), "
               +"id_vendedor VARCHAR(130), FOREIGN KEY (id_vendedor) REFERENCES vendedor(id_vendedor))")

# 2. Scrapping de datos para llenar nuestra base de datos Yapo

In [ ]:
from urllib.request import urlopen as uReq
import urllib.request

from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time

Con la librería Selenium, abriremos un navegador Chrome, en la página de los anuncios de Yapo en la región de Los Rios... __(UPDATE: todo chile)__

In [ ]:
browser = webdriver.Firefox()

myUrl = 'https://www.yapo.cl/chile/todos_los_avisos?ca=12_s&l=0'
#myUrl = 'https://www.yapo.cl/los_rios/todos_los_avisos?ca=11_s&l=0&w=1&cmn=243'
browser.get(myUrl)

Con la librería BeautifulSoup, realizamos un scrapping del código HTML para recuperar el enlace de la "última página"...

In [ ]:
pageSoup = soup(browser.page_source, 'html.parser')

pages = pageSoup.find('span',  {'class', 'nohistory FloatRight'}).a['href']

In [ ]:
pages

El parametro 'o' de la URL corresponde al número de la pagina en la lista de anuncios.

In [ ]:
index = pages.rfind('=')
print(index)
lastPage = int(pages[index+1:])
print(lastPage)
root_pages = pages[:index+1]
print(root_pages)

- la variable "index" permite identificar el indice del último caracter '='.

- la variable "lastPage" permite identificar el numero de la última página en la URL.

- la variable "root_pages" permite aislar la cadena de caracteres que corresponde a la raiz de la URL (sin el numero de página).


Empezaremos la iteración sobre cada página que escrapear...

In [ ]:
import datetime

def convert_date(yapo_date):
    month=yapo_date.split(" ")[0]
    year=yapo_date.split(" ")[1]
    
    convert = {'Enero' : 1,
            'Febrero' : 2,
            'Marzo' : 3,
            'Abril' : 4,
            'Mayo' : 5,
            'Junio' : 6,
            'Julio' : 7,
            'Agosto' : 8,
            'Septiembre' : 9, 
            'Octubre' : 10,
            'Noviembre' : 11,
            'Diciembre' : 12
           }
    
    
    new_date=datetime.date(int(year), convert[month], 1) 

    return new_date

print(convert_date("Febrero 2020"))

In [ ]:
for i in range(lastPage):
    
    #recuperarmos la URL de la pagina corriente
    url = root_pages + str(i+1)
    
    #con Selenium, vamos en navegar en esta página
    browser.get(url)
    
    #empezamos el scrapping de la pagina corriente
    pageSoup = soup(browser.page_source, 'html.parser')
    
    #recuperamos todos los tags HTML que corresponden a la lista de anuncios en esta pagina
    links = pageSoup.findAll('td', {'class' : 'thumbs_subject'})
    
    #empezamos a iterar sobre cada anuncio
    for link in links:
        
        #todos los datos que necesitamos encontrar
        url, titulo, descripcion, precio, fecha_publicacion, categoria="","","",None,"", ""
        nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion="","","","",""
        
        #Navegamos hacia la pagina del anuncio
        url=link.find('a',{'class':'title'})['href']
        print(url)
        browser.get(link.find('a',{'class':'title'})['href'])

        #RECUPERAMOS LA FECHA DE PUBLICACION DEL ANUNCIO          
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('time')):
            datetime_raw = pageSoup.find('time').attrs['datetime']
            date_publication_raw=datetime_raw.split("T")[0]
            date_publication_raw=  date_publication_raw.split("-")
            
            date_publication=datetime.date(int(date_publication_raw[0]), int(date_publication_raw[1]),
                                           int(date_publication_raw[2]))
            print("datetime:")
            print(date_publication)
        
        #RECUPERAMOS EL TITULO DEL ANUNCIO           
        pageSoup = soup(browser.page_source, 'html.parser')
        if(pageSoup.find('h1', {"id" : "da_subject"})):
            titulo = pageSoup.find('h1', {"id" : "da_subject"}).text.strip()
            print(titulo)
            
        #RECUPERAMOS LA DESCRIPCION DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "description"})):
            try:
                descripcion = pageSoup.find('div', {"class" : "description"}).text.split(' ', 1)[1].strip().replace(u'\n', u' ')
            except:
                continue
                
        #RECUPERAMOS EL PRECIO DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "price text-right"})):
            precio_raw = pageSoup.find('div', {"class" : "price text-right"}).text.strip().replace(u'\n', u' ').replace(u'\t', u'')
            precio_raw = precio_raw.split(" ")
            moneda = precio_raw[0]
            precio = precio_raw[1].split(",")[0].replace(u'.', u'')
            print("precio:"+precio)
            
        #RECUPERAMOS LA CATEGORIA DEL ANUNCIO
        if(pageSoup.find('div', {"class" : "breadcrumbs"})):
            categoria = pageSoup.find('div', {"class" : "breadcrumbs"}).find('a', {"id" : "breadcrumb_category"}).find('strong').text.strip().replace(u'\n', u' ')
            print(categoria)
            
        
        #RECUPERAMOS EL NOMBRE DEL VENDEDOR, SU FECHA DE INSCRIPCION Y SU LOCALIDAD
        if(pageSoup.find('aside', {"class" : "sidebar-right"})):
            aside = pageSoup.find('aside', {"class" : "sidebar-right"})
    
            #print(aside.find('seller-info'))
            
            #NOMBRE
            if(aside.find('seller-info')!=None):
                nombre=aside.find('seller-info').attrs['username']
            
                #FECHA DE INSCRIPCION
                fecha_inscripcion_raw=aside.find('seller-info').attrs['seniority']
                try:
                    fecha_inscripcion_raw=fecha_inscripcion_raw[len("En Yapo desde "):]
                    fecha_inscripcion=convert_date(fecha_inscripcion_raw)
                    print(fecha_inscripcion)
                except:
                    fecha_inscripcion=None
                    continue
                
                #LOCALIDAD
                localidad_raw=aside.find('seller-info').attrs['region']
            
                #print(localidad_raw)
            
                region_raw=localidad_raw.split(",")[0]
                ciudad_raw=localidad_raw.split(",")[1]
            
                codigo_region=region_raw.split(" ")[0]
                nombre_region=region_raw[len(codigo_region)+1:]
                ciudad=ciudad_raw[1:]
                
        # LLENAMOS LA BASE DE DATOS: TABLA VENDEDOR
        
        try:
            sql = "INSERT INTO vendedor (id_vendedor,nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion) VALUES (%s,%s, %s, %s,%s,%s)"
            val = (nombre+"_"+codigo_region+"_"+ciudad,nombre, ciudad, codigo_region, nombre_region, fecha_inscripcion)
            cursor.execute(sql, val)
        except Exception as e1:
            print(e1)
            #continue
        
        # LLENAMOS LA BASE DE DATOS: TABLA ANUNCIO
        try:
            sql = "INSERT INTO anuncio (url, titulo, descripcion, precio, moneda, fecha_publicacion, categoria, id_vendedor) VALUES (%s, %s, %s,%s, %s, %s, %s, %s)"
            val = (url, titulo, descripcion, int(precio), moneda, date_publication, categoria, nombre+"_"+codigo_region+"_"+ciudad)
            cursor.execute(sql, val)
        except Exception as e2:
            print(e2)
            #continue
        
        cursor.execute("COMMIT")

In [ ]:
...